<a href="https://colab.research.google.com/github/Xebec19/langchain-tutorials/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U langgraph langsmith langchain-community ollama

from langchain_community.llms import Ollama
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage

# Initialize the Ollama model
llm = Ollama(model="deepseek-r1")

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)

# Define the function that calls the model
def call_model(state: MessagesState):
    messages = state["messages"]
    response = llm.invoke(messages)
    return {"messages": messages + [AIMessage(content=response)]}

# Add node and edges
workflow.add_node("model", call_model)
workflow.add_edge(START, "model")
workflow.add_edge("model", END)  # Need to add END to complete the graph

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

# Add a thread_id
config = {"configurable": {"thread_id": "abc123"}}

# Restore the built-in input function if needed
import builtins
input = builtins.input

# Interactive loop
while True:
    query = input("Enter (type 'q' to quit): ")
    if query.lower() == "q":
        break

    input_messages = [HumanMessage(content=query)]
    output = app.invoke({"messages": input_messages}, config)

    # Print the last message
    if output["messages"]:
        print("\nAI Response:")
        output["messages"][-1].pretty_print()
    print()

Enter (type 'q' to quit): hello


ConnectionError: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/generate (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7a760f967790>: Failed to establish a new connection: [Errno 111] Connection refused'))